In [2]:
# E-Commerce Sales Performance Analysis
# Capstone Project - Business Analytics with AI

# ================================================
# 1. SETUP DAN IMPORT LIBRARIES
# ================================================

# Install required packages
!pip install plotly seaborn pandas numpy matplotlib scikit-learn

# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('default')
sns.set_palette("husl")

print("🚀 Libraries loaded successfully!")
print("📊 Ready for E-Commerce Sales Analysis")

# ================================================
# 2. DATA LOADING DAN EXPLORATION
# ================================================

# Option 1: Load from Kaggle (UK E-Commerce Data)
# Download: https://www.kaggle.com/datasets/carrie1/ecommerce-data

# For this demo, we'll create a sample dataset similar to real e-commerce data
# In real project, you would load: df = pd.read_csv('your_ecommerce_data.csv')

# Generate sample e-commerce data
np.random.seed(42)
n_records = 10000

# Create sample data
data = {
    'InvoiceNo': [f'INV{str(i).zfill(6)}' for i in range(1, n_records+1)],
    'StockCode': [f'SKU{np.random.randint(1000, 9999)}' for _ in range(n_records)],
    'Description': np.random.choice([
        'Laptop Gaming', 'Smartphone Premium', 'Headphone Wireless',
        'Smart Watch', 'Tablet Pro', 'Camera DSLR', 'Speaker Bluetooth',
        'Mouse Gaming', 'Keyboard Mechanical', 'Monitor 4K'
    ], n_records),
    'Quantity': np.random.poisson(3, n_records) + 1,
    'InvoiceDate': pd.date_range(start='2023-01-01', end='2024-01-01', periods=n_records),
    'UnitPrice': np.random.exponential(50, n_records) + 10,
    'CustomerID': np.random.randint(10000, 99999, n_records),
    'Country': np.random.choice(['Indonesia', 'Singapore', 'Malaysia', 'Thailand', 'Philippines'],
                               n_records, p=[0.4, 0.2, 0.2, 0.1, 0.1])
}

df = pd.DataFrame(data)
df['TotalAmount'] = df['Quantity'] * df['UnitPrice']

print("📁 Dataset Overview:")
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print("\n📋 First 5 rows:")
print(df.head())

print("\n📊 Dataset Info:")
df.info()

print("\n📈 Statistical Summary:")
print(df.describe())

# ================================================
# 3. DATA PREPROCESSING DAN CLEANING
# ================================================

print("\n🔧 DATA CLEANING & PREPROCESSING")
print("="*50)

# Check for missing values
print("Missing values:")
print(df.isnull().sum())

# Check for duplicates
print(f"\nDuplicate rows: {df.duplicated().sum()}")

# Remove negative quantities and prices (returns/errors)
print(f"Records before cleaning: {len(df)}")
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]
print(f"Records after cleaning: {len(df)}")

# Add time-based features
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month
df['Quarter'] = df['InvoiceDate'].dt.quarter
df['DayOfWeek'] = df['InvoiceDate'].dt.dayofweek
df['Hour'] = df['InvoiceDate'].dt.hour

# Add categorical features
df['PriceCategory'] = pd.cut(df['UnitPrice'],
                            bins=[0, 50, 100, 200, float('inf')],
                            labels=['Budget', 'Mid-range', 'Premium', 'Luxury'])

print("✅ Data preprocessing completed!")

# ================================================
# 4. EXPLORATORY DATA ANALYSIS (EDA)
# ================================================

print("\n📊 EXPLORATORY DATA ANALYSIS")
print("="*50)

# 4.1 Sales Trend Analysis
fig = make_subplots(rows=2, cols=2,
                    subplot_titles=['Monthly Sales Trend', 'Daily Sales Pattern',
                                   'Country Distribution', 'Product Performance'],
                    specs=[[{"type": "scatter"}, {"type": "bar"}],
                           [{"type": "pie"}, {"type": "bar"}]])

# Monthly sales
monthly_sales = df.groupby('Month')['TotalAmount'].sum()
fig.add_trace(go.Scatter(x=monthly_sales.index, y=monthly_sales.values,
                        mode='lines+markers', name='Monthly Sales'), row=1, col=1)

# Daily pattern
daily_sales = df.groupby('DayOfWeek')['TotalAmount'].mean()
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
fig.add_trace(go.Bar(x=days, y=daily_sales.values, name='Daily Average'), row=1, col=2)

# Country distribution
country_sales = df.groupby('Country')['TotalAmount'].sum().sort_values(ascending=False)
fig.add_trace(go.Pie(labels=country_sales.index, values=country_sales.values,
                    name='Country'), row=2, col=1)

# Product performance
product_sales = df.groupby('Description')['TotalAmount'].sum().sort_values(ascending=False).head(5)
fig.add_trace(go.Bar(x=product_sales.index, y=product_sales.values,
                    name='Top Products'), row=2, col=2)

fig.update_layout(height=800, title_text="E-Commerce Sales Dashboard", showlegend=False)
fig.show()

# 4.2 Customer Analysis
print("\n👥 CUSTOMER ANALYSIS")
customer_metrics = df.groupby('CustomerID').agg({
    'TotalAmount': ['sum', 'count', 'mean'],
    'Quantity': 'sum'
}).round(2)

customer_metrics.columns = ['Total_Spent', 'Order_Frequency', 'Avg_Order_Value', 'Total_Items']
customer_metrics = customer_metrics.sort_values('Total_Spent', ascending=False)

print("🏆 Top 10 Customers by Total Spending:")
print(customer_metrics.head(10))

# Customer Segmentation using RFM Analysis
print("\n📊 RFM ANALYSIS (Recency, Frequency, Monetary)")
reference_date = df['InvoiceDate'].max() + timedelta(days=1)

rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (reference_date - x.max()).days,  # Recency
    'InvoiceNo': 'nunique',  # Frequency
    'TotalAmount': 'sum'  # Monetary
}).round(2)

rfm.columns = ['Recency', 'Frequency', 'Monetary']

# Create RFM Scores
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5,4,3,2,1])
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 5, labels=[1,2,3,4,5])

rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)

print("RFM Analysis Summary:")
print(rfm.describe())

# ================================================
# 5. ADVANCED ANALYTICS & INSIGHTS
# ================================================

print("\n🧠 ADVANCED ANALYTICS & AI INSIGHTS")
print("="*50)

# 5.1 Sales Forecasting using Simple Moving Average
monthly_data = df.groupby(df['InvoiceDate'].dt.to_period('M'))['TotalAmount'].sum()
print("\n📈 Sales Forecasting (Simple Moving Average):")

# Calculate 3-month moving average
ma_3 = monthly_data.rolling(window=3).mean()
print("Last 3 months moving average:", ma_3.iloc[-1])

# 5.2 Market Basket Analysis (Association Rules)
print("\n🛒 MARKET BASKET ANALYSIS")
basket = df.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack(fill_value=0)
basket_sets = basket.applymap(lambda x: 1 if x > 0 else 0)

# Calculate support for each item
support = basket_sets.mean()
print("Top 5 Most Popular Products:")
print(support.sort_values(ascending=False).head())

# 5.3 Price Elasticity Analysis
price_elasticity = df.groupby('PriceCategory').agg({
    'Quantity': 'mean',
    'TotalAmount': 'sum',
    'CustomerID': 'nunique'
}).round(2)

print("\n💰 Price Category Performance:")
print(price_elasticity)

# 5.4 Seasonal Analysis
seasonal_data = df.groupby('Quarter')['TotalAmount'].agg(['sum', 'mean', 'count']).round(2)
print("\n🌍 Seasonal Performance:")
print(seasonal_data)

# ================================================
# 6. KEY INSIGHTS & FINDINGS
# ================================================

print("\n💡 KEY INSIGHTS & FINDINGS")
print("="*50)

# Calculate key metrics
total_revenue = df['TotalAmount'].sum()
total_customers = df['CustomerID'].nunique()
total_orders = df['InvoiceNo'].nunique()
avg_order_value = df.groupby('InvoiceNo')['TotalAmount'].sum().mean()
customer_lifetime_value = df.groupby('CustomerID')['TotalAmount'].sum().mean()

insights = f"""
🔍 BUSINESS INSIGHTS SUMMARY:

💰 Financial Performance:
   • Total Revenue: ${total_revenue:,.2f}
   • Average Order Value: ${avg_order_value:.2f}
   • Customer Lifetime Value: ${customer_lifetime_value:.2f}

👥 Customer Metrics:
   • Total Customers: {total_customers:,}
   • Total Orders: {total_orders:,}
   • Average Orders per Customer: {total_orders/total_customers:.1f}

📊 Top Performing Segments:
   • Best Country: {country_sales.index[0]} (${country_sales.iloc[0]:,.2f})
   • Best Product: {product_sales.index[0]} (${product_sales.iloc[0]:,.2f})
   • Peak Quarter: Q{seasonal_data['sum'].idxmax()} (${seasonal_data['sum'].max():,.2f})

🎯 Growth Opportunities:
   • Focus on {country_sales.index[1]} market (2nd largest)
   • Promote {product_sales.index[1]} (high potential)
   • Optimize {seasonal_data['sum'].idxmin()} quarter performance
"""

print(insights)

# ================================================
# 7. RECOMMENDATIONS & ACTION PLAN
# ================================================

print("\n🎯 STRATEGIC RECOMMENDATIONS")
print("="*50)

recommendations = """
📋 ACTIONABLE RECOMMENDATIONS:

1. 🚀 REVENUE OPTIMIZATION:
   → Increase focus on premium products (higher margins)
   → Implement dynamic pricing for peak hours/days
   → Cross-sell complementary products

2. 👥 CUSTOMER RETENTION:
   → Develop loyalty program for high-value customers
   → Re-engagement campaigns for low-recency customers
   → Personalized recommendations based on purchase history

3. 🌍 MARKET EXPANSION:
   → Invest in top-performing countries
   → Localize marketing for emerging markets
   → Optimize logistics for faster delivery

4. 📈 SEASONAL STRATEGY:
   → Prepare inventory for peak quarters
   → Promotional campaigns during low seasons
   → Seasonal product bundling

5. 🔧 OPERATIONAL EFFICIENCY:
   → Optimize supply chain for popular products
   → Improve customer service during peak times
   → Implement AI-powered demand forecasting

PRIORITY: Focus on top 20% customers (80/20 rule) for maximum ROI
"""

print(recommendations)

# ================================================
# 8. VISUALIZATION DASHBOARD
# ================================================

# Create comprehensive dashboard
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=['Revenue Trend', 'Customer Segments', 'Product Performance',
                   'Geographic Distribution', 'Price Analysis', 'Seasonal Patterns'],
    specs=[[{"type": "scatter"}, {"type": "pie"}],
           [{"colspan": 2, "type": "bar"}, None],
           [{"type": "bar"}, {"type": "heatmap"}]]
)

# Revenue trend with moving average
monthly_revenue = df.groupby('Month')['TotalAmount'].sum()
fig.add_trace(go.Scatter(x=monthly_revenue.index, y=monthly_revenue.values,
                        mode='lines+markers', name='Monthly Revenue'), row=1, col=1)

# Customer segments pie chart
rfm_segments = rfm.groupby(['R_Score', 'F_Score']).size().reset_index(name='count')
fig.add_trace(go.Pie(labels=[f"R{r}F{f}" for r,f in zip(rfm_segments['R_Score'], rfm_segments['F_Score'])],
                    values=rfm_segments['count'], name='Segments'), row=1, col=2)

# Product performance
product_perf = df.groupby('Description')['TotalAmount'].sum().sort_values(ascending=True).tail(8)
fig.add_trace(go.Bar(y=product_perf.index, x=product_perf.values,
                    orientation='h', name='Products'), row=2, col=1)

# Price category analysis
price_analysis = df.groupby('PriceCategory')['TotalAmount'].sum()
fig.add_trace(go.Bar(x=price_analysis.index, y=price_analysis.values,
                    name='Price Categories'), row=3, col=1)

# Seasonal heatmap
seasonal_heatmap = df.pivot_table(values='TotalAmount', index='Quarter',
                                 columns='Month', aggfunc='sum')
fig.add_trace(go.Heatmap(z=seasonal_heatmap.values,
                        x=[f"Month {i}" for i in seasonal_heatmap.columns],
                        y=[f"Q{i}" for i in seasonal_heatmap.index],
                        name='Seasonal'), row=3, col=2)

fig.update_layout(height=1200, title_text="🎯 E-Commerce Business Intelligence Dashboard")
fig.show()

# ================================================
# 9. EXPORT RESULTS
# ================================================

print("\n💾 EXPORTING RESULTS")
print("="*50)

# Save key datasets
rfm.to_csv('customer_rfm_analysis.csv')
customer_metrics.to_csv('customer_metrics.csv')
monthly_sales.to_csv('monthly_sales_trend.csv')

# Create summary report
summary_report = {
    'Analysis_Date': datetime.now().strftime('%Y-%m-%d'),
    'Total_Revenue': total_revenue,
    'Total_Customers': total_customers,
    'Total_Orders': total_orders,
    'Avg_Order_Value': avg_order_value,
    'Customer_Lifetime_Value': customer_lifetime_value,
    'Top_Country': country_sales.index[0],
    'Top_Product': product_sales.index[0],
    'Peak_Quarter': f"Q{seasonal_data['sum'].idxmax()}"
}

summary_df = pd.DataFrame([summary_report])
summary_df.to_csv('business_summary_report.csv', index=False)

print("✅ Files exported successfully:")
print("  • customer_rfm_analysis.csv")
print("  • customer_metrics.csv")
print("  • monthly_sales_trend.csv")
print("  • business_summary_report.csv")

print("\n🎉 ANALYSIS COMPLETED!")
print("="*50)
print("📊 Your E-Commerce Analytics Project is ready!")
print("🚀 Use these insights to drive business growth!")

🚀 Libraries loaded successfully!
📊 Ready for E-Commerce Sales Analysis
📁 Dataset Overview:
Shape: (10000, 9)
Columns: ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country', 'TotalAmount']

📋 First 5 rows:
   InvoiceNo StockCode          Description  Quantity  \
0  INV000001   SKU8270        Laptop Gaming         3   
1  INV000002   SKU1860          Camera DSLR         3   
2  INV000003   SKU6390           Tablet Pro         2   
3  INV000004   SKU6191  Keyboard Mechanical         4   
4  INV000005   SKU6734    Speaker Bluetooth         1   

                    InvoiceDate   UnitPrice  CustomerID    Country  \
0 2023-01-01 00:00:00.000000000  177.243698       92504  Indonesia   
1 2023-01-01 00:52:33.915391539   32.528779       22011  Indonesia   
2 2023-01-01 01:45:07.830783078   64.164370       90327   Malaysia   
3 2023-01-01 02:37:41.746174617   64.973166       92799  Singapore   
4 2023-01-01 03:30:15.661566156   12.931376       


👥 CUSTOMER ANALYSIS
🏆 Top 10 Customers by Total Spending:
            Total_Spent  Order_Frequency  Avg_Order_Value  Total_Items
CustomerID                                                            
62086           2686.93                1          2686.93            8
30964           2595.36                2          1297.68           12
92205           2331.86                2          1165.93           12
96238           2324.85                1          2324.85            6
12528           2324.06                1          2324.06            6
77511           2137.20                1          2137.20            7
34402           2134.21                2          1067.11           13
46807           2085.17                1          2085.17            8
79314           2063.00                1          2063.00            5
38871           2051.78                1          2051.78            8

📊 RFM ANALYSIS (Recency, Frequency, Monetary)
RFM Analysis Summary:
           Recency  


💾 EXPORTING RESULTS
✅ Files exported successfully:
  • customer_rfm_analysis.csv
  • customer_metrics.csv
  • monthly_sales_trend.csv
  • business_summary_report.csv

🎉 ANALYSIS COMPLETED!
📊 Your E-Commerce Analytics Project is ready!
🚀 Use these insights to drive business growth!


In [3]:
# Tambahkan cell baru di akhir notebook untuk download files:
from google.colab import files

print("📥 Downloading analysis results...")
files.download('customer_rfm_analysis.csv')
files.download('customer_metrics.csv')
files.download('monthly_sales_trend.csv')
files.download('business_summary_report.csv')
print("✅ All files downloaded successfully!")

📥 Downloading analysis results...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ All files downloaded successfully!
